# Setup

In [1]:
from idrtools import math
from twins_embedding import TwinsEmbeddingAnalysis
from matplotlib import pyplot as plt
from matplotlib import colors
from tqdm.notebook import tqdm
from astropy.table import Table
import utils
import numpy as np

/home/kyle/supernova/snfactory/manifold_twins/analysis/twins_embedding.py:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
%matplotlib inline

In [3]:
a = TwinsEmbeddingAnalysis()
a.settings['figure_directory'] = './output_generation/figures/'
a.settings['latex_directory'] = './output_generation/latex/'
a.run_analysis()

Loading dataset...
    IDR:          BLACKSTON
    Phase range: [-5.0, 5.0] days
    Bin velocity: 1000.0



Estimating the spectra at maximum light...
    Loaded cached stan model
    Using saved stan result
Reading between the lines...
    Loaded cached stan model
    Using saved stan result
Building masks...
    Masking 30/203 targets whose uncertainty power is 
    more than 0.100 of the intrinsic power.
Generating the manifold learning embedding...
Loading other indicators of diversity...
Fitting RBTL Twins Manifold GP...
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

# Attrition

In [ ]:
with a.latex_open('attrition_variables.tex') as f:
    utils.latex_command(f, 'numdatasetsne', '%d', a.attrition_enough_spectra)
    utils.latex_command(f, 'numdatasetspectra', '%d', a.attrition_total_spectra)
    utils.latex_print(f, "")
    utils.latex_command(f, 'nummanifoldsne', '%d', len(a.targets))
    utils.latex_command(f, 'nummanifoldspectra', '%d', len(a.spectra))
    utils.latex_command(f, 'numinterpsne', '%d', np.sum(a.uncertainty_mask))

In [ ]:
with a.latex_open('attrition_table.tex') as f:
    utils.latex_print(f, "\\textbf{General selection requirements} & \\\\")
    utils.latex_print(f, "Initial sample (SNe Ia with at least 5 SNfactory spectra) & %d \\\\" % a.attrition_enough_spectra)
    utils.latex_print(f, "SALT2 date of maximum light uncertainty < 1 day           & %d \\\\" % a.attrition_salt_daymax)
    utils.latex_print(f, "At least one spectrum within 5 days of maximum light      & %d \\\\" % a.attrition_range)
    utils.latex_print(f, "At least one spectrum with S/N 3300-3800~\AA\ > 100       & %d \\\\" % a.attrition_usable)
    utils.latex_print(f, "\hline")
    utils.latex_print(f, "\\textbf{Manifold learning selection requirements} & \\\\")
    utils.latex_print(f, r"\textbf{(Section~\ref{sec:isomap_sample})} & \\")
    utils.latex_print(f, "Spectrum at max. uncertainty < 10\\%% of intrinsic variance        & %d \\\\" % np.sum(a.uncertainty_mask))

# Dump of the embedding

In [ ]:
rbtl_mag_errs, rbtl_color_errs = a._calculate_rbtl_uncertainties()
pv_errs = a.calculate_peculiar_velocity_uncertainties(a.redshifts)
mag_errs = np.sqrt(rbtl_mag_errs**2 + pv_errs**2)

data_table = Table({
    'name': [a.iau_name_map.get(i.name, i.name).replace('_', '\_') for i in a.targets],
    # 'SALT2 $x_1$': ['%.2f $\pm$ %.2f' % (i, j) for i, j in zip(a.salt_x1, a.salt_fits['x1_err'])],
    # 'SALT2 $color$': ['%.2f $\pm$ %.2f' % (i, j) for i, j in zip(a.salt_colors, a.salt_fits['c_err'])],
    'SALT2 $x_1$': ['%.2f' % i for i in a.salt_x1],
    'SALT2 $x_1$ uncertainty': ['%.2f' % i for i in a.salt_fits['x1_err']],
    'SALT2 $c$': ['%.3f' % i for i in a.salt_colors],
    'SALT2 $c$ uncertainty': ['%.3f' % i for i in a.salt_fits['c_err']],
    'RBTL $A_V$': ['%.3f' % i for i in a.rbtl_colors],
    'RBTL $A_V$ uncertainty': ['%.3f' % i for i in rbtl_color_errs],
    'RBTL $\Delta M$': ['%.3f' % i for i in a.rbtl_mags],
    'RBTL $\Delta M$ uncertainty': ['%.3f' % i for i in mag_errs],
    'Twins Embedding Coordinate 1': ['%.3f' % i for i in a.embedding[:, 0]],
    'Twins Embedding Coordinate 2': ['%.3f' % i for i in a.embedding[:, 1]],
    'Twins Embedding Coordinate 3': ['%.3f' % i for i in a.embedding[:, 2]],
})

def dump_data(f, start=None, end=None):
    for row in data_table[start:end]:
        print(' & '.join(row) + ' \\\\', file=f)

with a.latex_open('twins_manifold_coordinates.tex') as f:
    dump_data(f)

with a.latex_open('twins_manifold_coordinates_short.tex') as f:
    dump_data(f, 95, 105)

# Dump of the spectra

In [7]:
import os

outdir = './data_release/'

os.makedirs(outdir, exist_ok=True)

readme_txt = """Data release for the Twins Embedding analysis (Boone et al. 2020a, 2020b)

Contact: Kyle Boone <kyboone@uw.edu>

These files contain the spectra of each of the supernovae used in the Twins Embedding analysis.
The spectra are provided in the restframe after they have been shifted to maximum light
and dereddened using the Reading Between the Lines (RBTL) algorithm.

Each file contains the data for a single supernova, with three columns of wavelength,
flux, and flux uncertainty.
"""

with open(f'{outdir}/README.txt', 'w') as f:
    print(readme_txt, file=f)

for idx, target in enumerate(a.targets):
    target_flux = a.scale_flux[idx]
    target_fluxerr = a.scale_fluxerr[idx]
    with open(f'{outdir}/{target}.dat', 'w') as f:
        print(' Wavelength            Flux     Uncertainty', file=f)
        for wave, flux, fluxerr in zip(a.wave, target_flux, target_fluxerr):
            print(f'{wave:11f} {flux:15e} {fluxerr:15e}', file=f)